In [1]:
# =============================================================================
# NOTEBOOK SETUP - Two Population Delay Sweep Execution
# =============================================================================
import os
import sys
from pathlib import Path

# Robust directory setup
if Path.cwd().name == 'two_populations':
    os.chdir("../../")

print(f"Working directory: {Path.cwd()}")
print(f"Python path: {sys.path[0]}")

Working directory: /home/ubuntu-deduce/Projects/izhikevich
Python path: /home/ubuntu-deduce/.asdf/installs/python/3.10.13/lib/python310.zip


In [2]:
import os
import pickle
import numpy as np
from typing import Dict, List

def inspect_pickle_structure(sweep_dir: str, config_name: str, trial_idx: int = 0):
    """Inspeccionar contenido de un pickle específico"""
    
    config_dir = os.path.join(sweep_dir, "figures", config_name)
    trial_file = os.path.join(config_dir, f"trial_{trial_idx+1}_full.pkl")
    
    print(f"\n{'='*60}")
    print(f"INSPECTING: {trial_file}")
    print(f"{'='*60}")
    
    if not os.path.exists(trial_file):
        print(f"❌ FILE NOT FOUND: {trial_file}")
        print(f"   Directory exists: {os.path.exists(config_dir)}")
        if os.path.exists(config_dir):
            files = os.listdir(config_dir)
            print(f"   Available files: {files[:10]}")
        return None
    
    try:
        with open(trial_file, 'rb') as f:
            data = pickle.load(f)
        
        print(f"✓ Pickle loaded successfully")
        print(f"\n--- KEYS IN PICKLE ---")
        print(f"Total keys: {len(data.keys())}")
        
        for key in sorted(data.keys()):
            value = data[key]
            
            # Analizar tipo y tamaño
            if isinstance(value, np.ndarray):
                shape = value.shape
                dtype = value.dtype
                size_mb = value.nbytes / (1024**2)
                non_zero = np.count_nonzero(value)
                print(f"  {key:20s} → ndarray {shape} {dtype} ({size_mb:.2f} MB) [{non_zero} non-zero]")
                
                # Mostrar preview si es 1D pequeño
                if len(shape) == 1 and shape[0] < 10:
                    print(f"      Preview: {value}")
                elif len(shape) == 1 and shape[0] > 0:
                    print(f"      Range: [{value.min():.3f}, {value.max():.3f}], mean={value.mean():.3f}")
                    
            elif isinstance(value, dict):
                print(f"  {key:20s} → dict with {len(value)} keys: {list(value.keys())[:5]}")
                
            elif isinstance(value, (int, float)):
                print(f"  {key:20s} → {type(value).__name__} = {value}")
                
            else:
                print(f"  {key:20s} → {type(value).__name__}")
        
        # Verificar claves críticas para plotting
        print(f"\n--- CRITICAL KEYS FOR PLOTTING ---")
        critical_keys = ['rate_A', 'rate_B', 'time', 'spike_times_A', 'spike_times_B']
        
        for key in critical_keys:
            if key in data:
                value = data[key]
                if isinstance(value, np.ndarray):
                    status = "✓ OK" if len(value) > 100 else f"⚠ SHORT ({len(value)} elements)"
                    print(f"  {key:20s} {status}")
                else:
                    print(f"  {key:20s} ❌ NOT AN ARRAY (type: {type(value).__name__})")
            else:
                print(f"  {key:20s} ❌ MISSING")
        
        return data
        
    except Exception as e:
        print(f"❌ ERROR LOADING PICKLE: {e}")
        import traceback
        traceback.print_exc()
        return None


def diagnose_all_configs(sweep_dir: str, results_db: Dict):
    """Diagnosticar todos los configs del sweep"""
    
    print(f"\n{'='*80}")
    print(f"DIAGNOSING ALL CONFIGS IN: {sweep_dir}")
    print(f"{'='*80}")
    
    configs = list(results_db.keys())
    print(f"\nTotal configs: {len(configs)}")
    
    summary = {
        'configs_with_pickles': 0,
        'configs_without_pickles': 0,
        'pickles_with_data': 0,
        'pickles_empty': 0,
        'missing_rate_A': [],
        'missing_rate_B': [],
        'short_timeseries': []
    }
    
    for config_name in configs:
        config_dir = os.path.join(sweep_dir, "figures", config_name)
        trial_file = os.path.join(config_dir, "trial_1_full.pkl")
        
        if not os.path.exists(trial_file):
            summary['configs_without_pickles'] += 1
            print(f"  ❌ {config_name}: no pickle found")
            continue
        
        summary['configs_with_pickles'] += 1
        
        try:
            with open(trial_file, 'rb') as f:
                data = pickle.load(f)
            
            # Check critical keys
            has_rate_A = 'rate_A' in data
            has_rate_B = 'rate_B' in data
            
            if not has_rate_A:
                summary['missing_rate_A'].append(config_name)
            if not has_rate_B:
                summary['missing_rate_B'].append(config_name)
            
            if has_rate_A and has_rate_B:
                len_A = len(data['rate_A']) if isinstance(data['rate_A'], np.ndarray) else 0
                len_B = len(data['rate_B']) if isinstance(data['rate_B'], np.ndarray) else 0
                
                if len_A > 100 and len_B > 100:
                    summary['pickles_with_data'] += 1
                    status = "✓"
                elif len_A == 0 or len_B == 0:
                    summary['pickles_empty'] += 1
                    status = "❌ EMPTY"
                else:
                    summary['short_timeseries'].append(config_name)
                    status = f"⚠ SHORT (A={len_A}, B={len_B})"
                    
                print(f"  {status} {config_name}")
            else:
                summary['pickles_empty'] += 1
                print(f"  ❌ INCOMPLETE {config_name}")
                
        except Exception as e:
            print(f"  ❌ ERROR {config_name}: {e}")
    
    # Print summary
    print(f"\n{'='*80}")
    print(f"SUMMARY")
    print(f"{'='*80}")
    print(f"Configs with pickles:     {summary['configs_with_pickles']}/{len(configs)}")
    print(f"Configs without pickles:  {summary['configs_without_pickles']}/{len(configs)}")
    print(f"Pickles with data (>100): {summary['pickles_with_data']}")
    print(f"Pickles empty:            {summary['pickles_empty']}")
    print(f"Pickles with short data:  {len(summary['short_timeseries'])}")
    
    if summary['missing_rate_A']:
        print(f"\nConfigs missing rate_A: {summary['missing_rate_A']}")
    if summary['missing_rate_B']:
        print(f"Configs missing rate_B: {summary['missing_rate_B']}")
    if summary['short_timeseries']:
        print(f"Configs with short timeseries: {summary['short_timeseries']}")
    
    return summary


def fix_load_raw_timeseries_with_logging():
    """Versión corregida con logging detallado"""
    
    code = '''
def load_raw_timeseries(sweep_dir, config_name, trial_idx=0):
    """Cargar datos raw con debugging mejorado"""
    
    config_dir = os.path.join(sweep_dir, "figures", config_name)
    trial_file = os.path.join(config_dir, f"trial_{trial_idx+1}_full.pkl")
    
    logger.info(f"[LOAD] Buscando: {trial_file}")
    
    if not os.path.exists(trial_file):
        logger.warning(f"[LOAD] ❌ No encontrado: {trial_file}")
        logger.warning(f"[LOAD]    Dir exists: {os.path.exists(config_dir)}")
        if os.path.exists(config_dir):
            files = [f for f in os.listdir(config_dir) if 'trial' in f]
            logger.warning(f"[LOAD]    Available trial files: {files[:5]}")
        return None
    
    try:
        with open(trial_file, 'rb') as f:
            data = pickle.load(f)
        
        logger.info(f"[LOAD] ✓ Pickle cargado: {len(data)} keys")
        
        # Verificar claves críticas
        critical_keys = ['rate_A', 'rate_B', 'time']
        missing = [k for k in critical_keys if k not in data]
        
        if missing:
            logger.warning(f"[LOAD] ⚠ Faltan claves: {missing}")
            logger.warning(f"[LOAD]    Claves disponibles: {list(data.keys())[:15]}")
            return None
        
        # Verificar longitud
        len_A = len(data['rate_A']) if isinstance(data['rate_A'], np.ndarray) else 0
        len_B = len(data['rate_B']) if isinstance(data['rate_B'], np.ndarray) else 0
        len_t = len(data['time']) if isinstance(data['time'], np.ndarray) else 0
        
        logger.info(f"[LOAD]    rate_A: {len_A} points, rate_B: {len_B} points, time: {len_t} points")
        
        if len_A < 100 or len_B < 100:
            logger.warning(f"[LOAD] ⚠ Time series muy cortas: A={len_A}, B={len_B}")
            return None
        
        logger.info(f"[LOAD] ✓ Datos válidos cargados")
        return data
        
    except Exception as e:
        logger.error(f"[LOAD] ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None
    '''
    
    return code


# ===================================================================
# SCRIPT DE USO
# ===================================================================


# Uso ejemplo:

# 1. Diagnosticar un pickle específico
sweep_dir = "./results/experiments/two_populations/step_input_diff_rois/20251006_172926/MS_LOGNORMAL_SEMISTRONG_k3p5_dt0p1_T4000_20trials__"  # TU PATH
config_name = "MS_SEMISTRONG_step_input_lognorm_1p1-0p3"  # TU CONFIG

data = inspect_pickle_structure(sweep_dir, config_name, trial_idx=0)

# 2. Diagnosticar todos los configs
# results_db = load_results_from_csvs(sweep_dir, config_names)
# summary = diagnose_all_configs(sweep_dir, results_db)


INSPECTING: ./results/experiments/two_populations/step_input_diff_rois/20251006_172926/MS_LOGNORMAL_SEMISTRONG_k3p5_dt0p1_T4000_20trials__/figures/MS_SEMISTRONG_step_input_lognorm_1p1-0p3/trial_1_full.pkl
✓ Pickle loaded successfully

--- KEYS IN PICKLE ---
Total keys: 17
  autocorr_A           → dict with 5 keys: ['lags', 'correlation', 'peak_lag', 'peak_value', 'lag_sign_convention']
  autocorr_B           → dict with 5 keys: ['lags', 'correlation', 'peak_lag', 'peak_value', 'lag_sign_convention']
  coherence            → dict with 9 keys: ['freqs', 'coherence', 'peak_freq', 'peak_coherence', 'theta_coherence']
  cross_correlation    → dict with 5 keys: ['lags', 'correlation', 'peak_lag', 'peak_value', 'lag_sign_convention']
  int_A                → dict with 3 keys: ['tau', 'quality', 'fit_quality']
  int_B                → dict with 3 keys: ['tau', 'quality', 'fit_quality']
  plv_pli              → dict with 5 keys: ['theta', 'alpha', 'beta', 'gamma', 'broadband']
  psd_A         

In [11]:
# =============================================================================
# BRIAN2 SETUP
# =============================================================================
from brian2 import *

# Performance optimization for standalone compilation
os.environ['CXXFLAGS'] = '-fopenmp -O3'
os.environ['LDFLAGS'] = '-lgomp'

# Clean slate
restore_initial_state()

# =============================================================================
# DEVICE CONFIGURATION
# =============================================================================
# Options: 'runtime', 'cpp_standalone', 'cuda_standalone'
DEVICE_MODE = 'runtime'  # Change this for performance

# Hardware detection
import multiprocessing
N_CORES = multiprocessing.cpu_count()
print(f"Available CPU cores: {N_CORES}")

if DEVICE_MODE == 'cpp_standalone':
    set_device('cpp_standalone', directory='brian2_output', build_on_run=True)
    prefs.devices.cpp_standalone.openmp_threads = min(N_CORES, 20)  # Adaptive
    print(f"[DEVICE] cpp_standalone with {prefs.devices.cpp_standalone.openmp_threads} threads")

elif DEVICE_MODE == 'cuda_standalone':
    try:
        # from brian2cuda import *  # Uncomment if available
        set_device('cuda_standalone', directory='brian2_cuda_output', build_on_run=False)
        print("[DEVICE] cuda_standalone enabled")
    except ImportError:
        print("[WARNING] brian2cuda not available, falling back to cpp_standalone")
        set_device('cpp_standalone', directory='brian2_output', build_on_run=True)
        
else:  # runtime
    print(f"[DEVICE] runtime mode (single-threaded)")

print(f"Brian2 device: {get_device()}")
print("=" * 60)

Available CPU cores: 12
[DEVICE] runtime mode (single-threaded)
Brian2 device: <brian2.devices.device.RuntimeDevice object at 0x7f951ffaddb0>


In [ ]:
from src.two_populations.plots.basic_plots import plot_raster_results
from src.two_populations.model import IzhikevichNetwork
from src.two_populations.metrics import analyze_simulation_results
from src.two_populations.helpers.helpers import load_results_from_csvs, extract_delay_range_from_config, load_raw_timeseries

# ==== CREAR CARPETA DEL SWEEP Y GUARDAR CONFIG ====
from src.two_populations.sweep import (
    create_sweep_folder, save_config_json, delay_distribution_sweep,
    save_results_with_csv, save_results
)

# ==== CONFIG BÁSICA DEL SWEEP ====
from datetime import datetime
import os, json
from copy import deepcopy
import multiprocessing

from src.two_populations.plots.plots_sweeps import plot_delay_distribution_comparison

def run_single_config(args):
    config_name, delay_config, sweep_dir, dt, time, base_params, n_trials, device_mode, directionality = args
    single_config = {config_name: delay_config}
    
    print(f"Config params: {args=}")
    
    results_db = delay_distribution_sweep(
        sweep_dir, dt, time, IzhikevichNetwork,
        base_params, single_config, 
        directionality=directionality,
        n_trials=n_trials, 
        device_mode=device_mode
    )
    
    paths = save_results_with_csv(
        results_db, 
        out_dir=os.path.join(sweep_dir, f"config_{config_name}"),
        tag=config_name,
        per_trial_csv=True
    )
    
    return config_name

names = ['WEAK', 'MEDIUM', 'SEMISTRONG', 'STRONG']
values = [0.2, 0.5, 1.0, 2.0]

for i in range(0,1):
    
    TAG = f"MS_BETA_MULTI_SAME_NOISE_CI_STEP_{names[i]}_{str(values[i]).replace('.','p')}_k3p5_dt0p1_T4000_30trials"
    DT_MS = 0.1
    T_MS  = 4000
    N_TRIALS = 30
    DEVICE_MODE = "runtime"
    DIRECTIONALITY = 0

    # tus params (tal cual los defines)
    k_factor = 3.5
    noise_int = 5.1*1.2
    noise_inh_factor = 0.451
    inter_k_factor = values[i]

    base_params = {
        'pop_A': {'Ne': 800,'Ni': 200,'k_exc': k_factor*0.5,'k_inh': k_factor*1.0,
                'noise_exc': noise_int,'noise_inh': round(noise_int*noise_inh_factor, 2),
                'p_intra': 0.1,'delay': 0.2,'noise_type': 'poisson', 'step': True},
        'pop_B': {'Ne': 800,'Ni': 200,'k_exc': k_factor*0.5,'k_inh': k_factor*1.0,
                'noise_exc': noise_int*0.8,'noise_inh': round(noise_int*noise_inh_factor, 2)*0.8,
                'p_intra': 0.1,'delay': 0.2,'noise_type': 'poisson', 'step': False},
        'connection': {'p_inter': 0.02,'weight_scale': k_factor*inter_k_factor}
    }


    delay_configs = {
        f'MS_{names[i]}_coupling_step_input_uniform_0-6': {'type': 'uniform', 'params': {'low': 0.0, 'high': 6.0}},
        f'MS_{names[i]}_coupling_step_input_uniform_0p75-5p25': {'type': 'uniform', 'params': {'low': 0.75, 'high': 5.25}},
        f'MS_{names[i]}_coupling_step_input_uniform_1p5-4p5': {'type': 'uniform', 'params': {'low': 1.5, 'high': 4.5}},
        f'MS_{names[i]}_coupling_step_input_uniform_2p25-p75': {'type': 'uniform', 'params': {'low': 2.25, 'high': 3.75}},
    }
        
    # delay_configs = {
    #     # === FORMAS EXTREMAS ===
    #     # Forma de U - alta variabilidad (conexiones muy rápidas o muy lentas)
    #     f'MS_{names[i]}_coupling_beta_U_shape': {'type': 'beta', 'params': {'alpha': 0.5, 'beta': 0.5, 'scale': 4.0}},
        
    #     # Uniforme - todos los retrasos igualmente probables
    #     f'MS_{names[i]}_coupling_beta_uniform': {'type': 'beta', 'params': {'alpha': 1.0, 'beta': 1.0, 'scale': 4.0}},
        
    #     # === ASIMÉTRICAS (SESGADAS) ===
    #     # Sesgada hacia retrasos cortos - la mayoría de conexiones son rápidas
    #     f'MS_{names[i]}_coupling_beta_fast_bias': {'type': 'beta', 'params': {'alpha': 2.0, 'beta': 8.0, 'scale': 4.0}},
    #     f'MS_{names[i]}_coupling_beta_very_fast': {'type': 'beta', 'params': {'alpha': 1.0, 'beta': 4.0, 'scale': 4.0}},
        
    #     # Sesgada hacia retrasos largos - dominan las conexiones lentas  
    #     f'MS_{names[i]}_coupling_beta_slow_bias': {'type': 'beta', 'params': {'alpha': 8.0, 'beta': 2.0, 'scale': 4.0}},
    #     f'MS_{names[i]}_coupling_beta_very_slow': {'type': 'beta', 'params': {'alpha': 4.0, 'beta': 1.0, 'scale': 4.0}},
        
    #     # === SIMÉTRICAS UNIMODALES ===
    #     # Concentrada en el centro - retrasos intermedios más probables
    #     f'MS_{names[i]}_coupling_beta_centered_narrow': {'type': 'beta', 'params': {'alpha': 4.0, 'beta': 4.0, 'scale': 4.0}},
    #     f'MS_{names[i]}_coupling_beta_centered_sharp': {'type': 'beta', 'params': {'alpha': 8.0, 'beta': 8.0, 'scale': 4.0}},
    #     f'MS_{names[i]}_coupling_beta_centered_very_sharp': {'type': 'beta', 'params': {'alpha': 12.0, 'beta': 12.0, 'scale': 4.0}},
        
    #     # === CASOS INTERMEDIOS ===
    #     # Ligeramente sesgadas - sutiles preferencias direccionales
    #     f'MS_{names[i]}_coupling_beta_mild_fast': {'type': 'beta', 'params': {'alpha': 3.0, 'beta': 5.0, 'scale': 4.0}},
    #     f'MS_{names[i]}_coupling_beta_mild_slow': {'type': 'beta', 'params': {'alpha': 5.0, 'beta': 3.0, 'scale': 4.0}},
        
    #     # === DIFERENTES ESCALAS TEMPORALES ===
    #     # Misma forma, diferentes rangos de retrasos
    #     f'MS_{names[i]}_coupling_beta_short_range': {'type': 'beta', 'params': {'alpha': 4.0, 'beta': 4.0, 'scale': 2.0}},
    #     f'MS_{names[i]}_coupling_beta_long_range': {'type': 'beta', 'params': {'alpha': 4.0, 'beta': 4.0, 'scale': 8.0}},
    # }

    # delay_configs = {
    #     f'MS_{names[i]}_coupling_step_input_gauss_12-4': {'type': 'gaussian', 'params': {'mu': 12.0, 'sigma': 4.0}},
    #     f'MS_{names[i]}_coupling_step_input_gauss_12-8': {'type': 'gaussian', 'params': {'mu': 12.0, 'sigma': 8.0}},
    #     f'MS_{names[i]}_coupling_step_input_gauss_12-12': {'type': 'gaussian', 'params': {'mu': 12.0, 'sigma': 12.0}},
    #     f'MS_{names[i]}_coupling_step_input_gauss_12-16': {'type': 'gaussian', 'params': {'mu': 12.0, 'sigma': 16.0}},
    #     f'MS_{names[i]}_coupling_step_input_gauss_12-20': {'type': 'gaussian', 'params': {'mu': 12.0, 'sigma': 20.0}},
    #     f'MS_{names[i]}_coupling_step_input_gauss_12-24': {'type': 'gaussian', 'params': {'mu': 12.0, 'sigma': 24.0}},
    # }

    # delay_configs = {
    #     f'MS_{names[i]}_coupling_step_input_delta_0': {'type': 'constant', 'value': 0.0},
    #     f'MS_{names[i]}_coupling_step_input_delta_4': {'type': 'constant', 'value': 4.0},
    #     f'MS_{names[i]}_coupling_step_input_delta_8': {'type': 'constant', 'value': 8.0},
    #     f'MS_{names[i]}_coupling_step_input_delta_12': {'type': 'constant', 'value': 12.0},
    #     f'MS_{names[i]}_coupling_step_input_delta_16': {'type': 'constant', 'value': 16.0},
    #     f'MS_{names[i]}_coupling_step_input_delta_20': {'type': 'constant', 'value': 20.0},
    # }

    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    sweep_dir = create_sweep_folder(
        base_dir="./results/experiments/two_populations/step_input",
        tag=TAG, timestamp=ts
    )

    config_dict = {
        "tag": TAG, "timestamp": ts,
        "device_mode": DEVICE_MODE,
        "dt_ms": DT_MS, "T_total_ms": T_MS,
        "n_trials": N_TRIALS, "directionality": DIRECTIONALITY,
        "base_params": deepcopy(base_params),
        "delay_configs": deepcopy(delay_configs),
        "seeds_variable": [50+i for i in range(N_TRIALS)],
        "seed_fixed": 42,
        "control_type": "strict"
    }
    save_config_json(config_dict, os.path.join(sweep_dir, "config.json"))


    # ==== CORRER SWEEP CON MULTIPROCESSING OPTIMIZADO ====
    config_args = [
        (name, config, sweep_dir, DT_MS, T_MS, base_params, N_TRIALS, DEVICE_MODE, DIRECTIONALITY)
        for name, config in delay_configs.items()
    ]

    # En el proceso principal:
    with multiprocessing.Pool(3) as pool:
        completed_configs = pool.map_async(run_single_config, config_args).get(timeout=1800)

    print(f" --- COMPLETED: {completed_configs}")
    
    # Reconstruir datos desde CSVs para plot de comparación
    results_db_light = load_results_from_csvs(sweep_dir, completed_configs)

    fig = plot_delay_distribution_comparison(results_db_light, save_path=os.path.join(sweep_dir, "comparison.png"))
    
    print(f"Sweep folder: {sweep_dir}")

In [ ]:
from src.two_populations.plots import load_trials_data_for_dashboard, plot_advanced_metrics_dashboard
import os

sweep_dir = './results/experiments/two_populations/step_input/MS_UNIFORM_1_SAME_NOISE_CI_STEP_MEDIUM_0p5_k3p5_dt0p1_T4000_30trials__20250919_183201'

config_dirs = [d for d in os.listdir(sweep_dir) if d.startswith('config_')]
completed_configs = [d.replace('config_', '') for d in config_dirs]

results_db = load_results_from_csvs(sweep_dir, completed_configs, filter_outliers=True)
trials_data = load_trials_data_for_dashboard(sweep_dir, completed_configs)

fig = plot_advanced_metrics_dashboard(results_db, trials_data)

In [ ]:
from src.two_populations.plots import extract_delay_parameters, extract_metrics_data, plot_correlation_analysis, print_correlation_summary

delay_df = extract_delay_parameters(results_db)
metrics_df = extract_metrics_data(results_db)
fig, corr_matrix, p_matrix = plot_correlation_analysis(delay_df, metrics_df)
print_correlation_summary(corr_matrix, p_matrix)

In [ ]:
import psutil
print(f"RAM disponible: {psutil.virtual_memory().available / 1e9:.1f} GB")
print(f"RAM total: {psutil.virtual_memory().total / 1e9:.1f} GB")

In [ ]:
import brian2
print(f"Brian2 version: {brian2.__version__}")
print(f"Device: {brian2.get_device()}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ==== setup ====
time = 3500
dt_ms = 0.1
n_timesteps = int(time/dt_ms)
n_neurons = 800

# “estado sano” como referencia
lambda_exc = 5.1**2 / dt_ms
lambda_inh = 2.0**2 / dt_ms

def step_profile(n, start=0.25, end=0.75, base=0.0, elev=1.0):
    prof = np.full(n, base, dtype=float); s,e = int(n*start), int(n*end)
    prof[s:e] = elev; return prof

# perfiles (μ = drive DC ; σ = factor de varianza)
# mu_exc    = step_profile(n_timesteps, 0.25, 0.75, base=0.7*lambda_exc,  elev=1.15*lambda_exc)
# sigma_exc = step_profile(n_timesteps, 0.25, 0.75, base=0.9,            elev=1.6)

# mu_inh    = step_profile(n_timesteps, 0.25, 0.75, base=0.6*lambda_inh, elev=0.9*lambda_inh)
# sigma_inh = step_profile(n_timesteps, 0.25, 0.75, base=0.9,            elev=1.2)

mu_exc    = step_profile(n_timesteps, 0.25, 0.75, base=0.0,  elev=0.0)
sigma_exc = step_profile(n_timesteps, 0.25, 0.75, base=1.0,   elev=2.0)

mu_inh    = step_profile(n_timesteps, 0.25, 0.75, base=0.0, elev=0.0)
sigma_inh = step_profile(n_timesteps, 0.25, 0.75, base=1.0,  elev=1.0)

# ruido centrado (z ~ var≈1) y composición I = μ + σ·z
# z_exc = (np.random.poisson(lambda_exc, (n_timesteps, n_neurons)) - 0*lambda_exc)#/np.sqrt(lambda_exc)
# z_inh = (np.random.poisson(lambda_inh, (n_timesteps, n_neurons)) - 0*lambda_inh)#/np.sqrt(lambda_inh)
z_exc = (np.random.randn(n_timesteps, n_neurons) - 0*lambda_exc)#/np.sqrt(lambda_exc)
z_inh = (np.random.randn(n_timesteps, n_neurons) - 0*lambda_inh)#/np.sqrt(lambda_inh)
I_exc = sigma_exc[:,None]*z_exc
I_inh =  sigma_inh[:,None]*z_inh

# ==== visualización ====
t = np.arange(n_timesteps)*dt_ms
neuron = 0
win = int(500/dt_ms)         # 500 ms
hop = max(1, win//4)

def rolling_std(x, w, h):
    out_t, out = [], []
    for i in range(0, len(x)-w, h):
        seg = x[i:i+w]
        out_t.append((i+w/2)*dt_ms)
        out.append(np.std(seg - seg.mean()))  # std centrada (solo variabilidad)
    return np.array(out_t), np.array(out)

fig, axes = plt.subplots(3,1, figsize=(12,8), sharex=False)

# (1) perfiles μ(t) y σ(t)
axes[0].plot(t, mu_exc,  label='μ_exc (DC)', lw=2)
axes[0].plot(t, mu_inh,  label='μ_inh (DC)', lw=2, ls='--')
axes[0].plot(t, sigma_exc*np.sqrt(lambda_exc), label='σ_exc·√λ_exc (escala var)', lw=1.5)
axes[0].plot(t, sigma_inh*np.sqrt(lambda_inh), label='σ_inh·√λ_inh (escala var)', lw=1.5, ls='--')
axes[0].set_title('Perfiles de modulación (media y variabilidad)')
axes[0].set_ylabel('Amplitud'); axes[0].grid(True, alpha=0.3); axes[0].legend()

# (2) neurona ejemplo: base vs modulado (excitatoria)
axes[1].plot(t, z_exc[:,neuron],  lw=0.6, alpha=0.7, label='z_exc (base, μ=0, σ≈1)')
axes[1].plot(t, I_exc[:,neuron], lw=0.8, alpha=0.9, label='I_exc = μ_exc + σ_exc·z')
axes[1].set_title(f'Neurona excitatoria #{neuron}: señal base vs modulada')
axes[1].set_ylabel('Entrada'); axes[1].grid(True, alpha=0.3); axes[1].legend()

# (3) variabilidad temporal (std centrada en ventana deslizante)
tb, std_base = rolling_std(z_exc[:,neuron], win, hop)
tm, std_mod  = rolling_std(I_exc[:,neuron], win, hop)
axes[2].plot(tb, std_base, lw=2, label='STD base (z centrado)')
axes[2].plot(tm, std_mod,  lw=2, label='STD modulada (μ+σ·z, centrada)')
axes[2].set_title('Evolución de la variabilidad (500 ms, solapamiento 75%)')
axes[2].set_xlabel('Tiempo (ms)'); axes[2].set_ylabel('STD'); axes[2].grid(True, alpha=0.3); axes[2].legend()

plt.tight_layout(); plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
plt.hist(np.random.poisson(2.0, (1000, 2))[:,0]+np.random.poisson(2.0, (1000, 2))[:,0])

== Estadísticas del Grupo A ===

=== INPUT TALÁMICO ===
Brian2 exc - Media por neurona: -0.0019
Brian2 exc - Std por neurona: 4.9974

=== INPUT SINÁPTICO ===
Brian2 exc - Media: 0.8016
Brian2 exc - Std: 0.5017
Brian2 exc - Rango típico: [-4.84, 9.81]

=== Estadísticas del Grupo B ===

=== INPUT TALÁMICO ===
Brian2 exc - Media por neurona: -0.0039
Brian2 exc - Std por neurona: 5.0036

=== INPUT SINÁPTICO ===
Brian2 exc - Media: 0.8246
Brian2 exc - Std: 0.5066
Brian2 exc - Rango típico: [-4.62, 6.72]

In [ ]:
peaks = []

for time in np.arange(0,2600,50):

    warmup = time

    results_dict = {}

    # Una línea para analizar
    results_dict = {
        'baseline': analyze_simulation_results(results['A']['spike_monitor'], results['B']['spike_monitor'], 1000, "Baseline", warmup=warmup)
    }
    
    peaks.append(results_dict['baseline']['cross_correlation']['peak_value'])

plt.plot(np.arange(0,2600,50), peaks, 'o-')